# Estudo de Preços e Combustíveis

Usando Cassandra, Python e Jupyter Notebook

### Declaração de classses de dados

In [ ]:
class Vendas():
    def __init__(self, regiao, estado, municipio, revenda, 
                 produto, dt_coleta, valor_compra, 
                 valor_venda, unidade_medida, bandeira):
        self.regiao = regiao
        self.estado = estado
        self.municipio = municipio
        self.revenda = revenda
        self.produto = produto
        self.dt_coleta = dt_coleta
        self.mes = int(dt_coleta[3:5])
        self.ano = int(dt_coleta[8:10])
        
        if ((self.valor_compra is None) or (self.valor_compra == '')):
            return None
        else:
            return float(self.valor_compra.replace(',', '.'))
        
        if ((self.valor_venda is None) or (self.valor_venda == '')):
            return None
        else:
            return float(self.valor_venda.replace(',', '.'))        
        
        self.unidade_medida = unidade_medida
        self.bandeira = bandeira

    def get_regiao(self):
        return self.regiao

    def get_estado(self):
        return self.estado

    def get_municipio(self):
        return self.municipio

    def get_revenda(self):
        return self.revenda

    def get_produto(self):
        return self.produto    
        
    def get_dt_coleta(self):
        return self.dt_coleta
    
    def get_mes(self):
        return self.mes
    
    def get_ano(self):
        return self.ano
    
    def get_valor_compra(self):
        return self.valor_compra
    
    def get_valor_venda(self):
        return valor_venda
    
    def get_unidade_medida(self):
        return self.unidade_medida        
    
    def get_bandeira(self):
        return self.bandeira     

In [ ]:
class Vendas_Valor():
    def __init__(self, municipio, bandeira, revenda, produto, dt_coleta, valor_venda):
        self.municipio = municipio
        self.bandeira = bandeira
        self.revenda = revenda
        self.produto = produto
        self.dt_coleta = dt_coleta
        self.valor_venda = valor_venda

    def get_municipio(self):
        return self.municipio
    
    def get_bandeira(self):
        return self.bandeira
    
    def get_revenda(self):
        return self.revenda    
    
    def get_produto(self):
        return self.produto    

    def get_dt_coleta(self):
        return self.dt_coleta
    
    def get_valor_venda(self):
        return self.valor_venda    

In [ ]:
class Dolares_Mes():
    def __init__(self, ano, mes, valor_compra):
        self.ano = ano
        self.mes = mes
        self.valor_compra = valor_compra

    def get_ano(self):
        return self.ano
    
    def get_mes(self):
        return self.mes
    
    def get_valor_compra(self):
        return self.valor_compra

In [ ]:
class Dolares_Mes():
    def __init__(self, ano, mes, produto):
        self.ano = ano
        self.mes = mes
        self.produto = produto
        self.qtd = qtd
        self.valor_venda = valor_venda        

    def get_ano(self):
        return self.ano
    
    def get_mes(self):
        return self.mes
    
    def get_produto(self):
        return self.produto
    
    def get_qtd(self):
        return self.qtd
    
    def get_valor_venda(self):
        return self.valor_venda

In [ ]:
class Media_Mes():
    def __init__(self, ano, mes, diesel, diesel_s10, etanol, gasolina, gnv):
        self.ano = ano
        self.mes = mes
        self.dolar = dolar
        self.diesel = diesel
        self.diesel_s10 = diesel_s10        
        self.etanol = etanol
        self.gasolina = gasolina        
        self.gnv = gnv        

    def get_ano(self):
        return self.ano
    
    def get_mes(self):
        return self.mes
    
    def get_dolar(self):
        return self.dolar
    
    def get_diesel(self):
        return self.diesel
    
    def get_diesel_s10(self):
        return self.diesel_s10
    
    def get_etanol(self):
        return self.etanol
    
    def get_gasolina(self):
        return self.gasolina
    
    def get_gnv(self):
        return self.gnv    

### Declaração de conexões e carregadores de dados

In [42]:
def load_vendas_info_file(fname):
    vendas = []
    with open(fname, encoding="latin-1") as f:
        next(f)
        for line in f: 
            vendas.append(Vendas(*str(line).strip().split('\t')))
    f.close()
    return vendas

In [43]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

CASSANDRA_USER = 'cassandra'
CASSANDRA_PASSWD = 'password'

def create_connection(nodeIP, keyspaceName):
    """
    Function to create a connection object
    """

    client = Connection()
    client.connect(nodeIP) #Observar que o nodeIP é uma lista e não somente um único valor
    client.set_keyspace(keyspaceName)
    return client


class Connection:
    session = None
    keyspace = ""

    def connect(self, nodes):
        auth_provider = PlainTextAuthProvider(username=str(CASSANDRA_USER), password=str(CASSANDRA_PASSWD))
        cluster = Cluster(nodes, auth_provider=auth_provider)
        metadata = cluster.metadata
        self.session = cluster.connect()

    def get_session():
        return self.session

    def set_keyspace(self, keyspace):
        self.keyspace = keyspace

    def get_keyspace(self):
        return self.keyspace

    def close(self):
        self.session.cluster.shutdown()
        self.session.shutdown()

    def query_schema(self, table):
        table_keyspace = self.keyspace+"."+table
        query = " SELECT * FROM "+ table_keyspace +";"
        result_set = self.session.execute(query)
        return result_set

In [44]:
#Classe pai das conexões
class CommonCassandraDB():

    def __init__(self):
        self.client = None

    def set_cassandra_client(self, nodeIP, keyspaceName):
        self.client = create_connection(nodeIP, keyspaceName)

    def close_connection(self):
        self.client.close()

### Declaração de classes com conexões

In [45]:
class VendasCassandra(CommonCassandraDB):
    
    def __init__(self, nodeIP, keyspace):
        self.cassandra_table_reference = 'vendas' #nome da Tabela do Cassandra

        self.set_cassandra_client(nodeIP, keyspace)
    
    def get_full_table(self):
        table = self.client.get_keyspace() + "." + self.cassandra_table_reference
        sql_query = """
                    SELECT *
                    FROM @$$$$$$$$$$$$@
                """
        sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
        result_set = self.client.session.execute(sql_query)
        return result_set 

    def show_row(self, row):
        print(row.get_regiao()) 
        print(row.get_estado())
        print(row.get_municipio()) 
        print(row.get_revenda())
        print(row.get_produto())
        print(row.get_dt_coleta())
        print(row.get_mes())
        print(row.get_ano())
        print(row.get_valor_compra())
        print(row.get_valor_venda())
        print(row.get_unidade_medida())
        print(row.get_bandeira())

    def insert(self, venda):
        try:
            table = self.client.get_keyspace() + "." + 'vendas'
            sql_query = """
            INSERT INTO @$$$$$$$$$$$$@
            (
                regiao, 
                estado, 
                municipio, 
                revenda, 
                produto, 
                dt_coleta, 
                mes,
                ano,
                valor_compra, 
                valor_venda, 
                unidade_medida, 
                bandeira
            )
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?);
            """
            sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
            bound_statement = self.client.session.prepare(sql_query)
            self.client.session.execute( bound_statement.bind((
                    venda.get_regiao(),
                    venda.get_estado(),
                    venda.get_municipio(),
                    venda.get_revenda(),
                    venda.get_produto(),
                    venda.get_dt_coleta(),
                    venda.get_mes(),
                    venda.get_ano(),
                    venda.get_valor_compra(), 
                    venda.get_valor_venda(), 
                    venda.get_unidade_medida(), 
                    venda.get_bandeira()                
                    ))
                )
            return True
        except:
            raise
        return False

### Carregar arquivos originais

[Segundo Semestre de 2016](http://www.anp.gov.br/images/dadosabertos/precos/2016-2_CA.csv)  
[Primeiro Semestre de 2017](http://www.anp.gov.br/images/dadosabertos/precos/2017-1_CA.csv)  
[Segundo Semestre de 2017](http://www.anp.gov.br/images/dadosabertos/precos/2017-2_CA.csv)  
[Primeiro Semestre de 2018](http://www.anp.gov.br/images/dadosabertos/precos/2018-1_CA.csv)  

In [ ]:
vendasDAO = VendasCassandra(['172.17.0.2'], 'gas_price')

for file in ['2016-2_CA.csv', '2017-1_CA.csv', '2017-2_CA.csv', '2018-1_CA.csv']:
    for venda in load_vendas_info_file(file):
        vendasDAO.insert(venda)

### Geração de dados para análise